In [1]:
import pandas as pd 
import numpy as np 

def moment(series,degree):
    n = len(series)
    moment = sum(x ** degree for x in series) / n
    return moment

def skewness(x):
    return(moment(x,3)/(np.std(x)**3))

def kurtosis(x):
    return(moment(x,4)/(np.std(x)**4))
    
def annualized_returns(df):

    num_years = len(df)/365  # 365 is the number of trading days in a year
    cumulative_returns = (df + 1).cumprod()
    annualized_returns = cumulative_returns.iloc[-1] ** (1 / num_years) - 1

    return annualized_returns

def annualized_volatility(returns):
    
    returns = np.array(returns)
    n = 365  # Assuming 365 trading days in a year
    volatility = np.std(returns) * np.sqrt(n)

    return volatility
    

def sharpe_ratio(returns, risk_free_rate):
    returns = np.array(returns)
    excess_returns = returns - risk_free_rate
    mean_return = np.mean(excess_returns)
    std_dev = np.std(excess_returns)

    if std_dev == 0:
        return 0.0

    sharpe_ratio = mean_return / std_dev
    return sharpe_ratio

def Jarque_Barque_test(x):
    s = skewness(x)
    k = kurtosis(x)
    return((len(x)/6)*(s**2+(k-3)**2)/4)

def drawdowns(returns):
    returns = np.array(returns)
    cumulative_returns = np.cumprod(1+returns)
    peaks = np.maximum.accumulate(cumulative_returns) #np.maximum.accumulate accumulates the result of applying the operator to all elements.
    drawdowns = (cumulative_returns - peaks) / peaks
    return drawdowns

def semi_deviation (returns):
    
    returns = np.array(returns)
    mean_return = np.mean(returns)
    negative_returns = returns[returns<mean_return]
    squared_negative_returns = np.square(negative_returns)
    semi_deviation = np.sqrt(np.mean(squared_negative_returns))

    return semi_deviation

def historical_VaR(returns,confidence_level):
    returns = np.array(returns)
    sorted_returns = np.sort(returns)
    index = int((1 - confidence_level) * len(sorted_returns))
    VaR = abs(sorted_returns[index])

    return VaR

def historical_CVaR(returns,confidence_level):
    returns = np.array(returns)
    sorted_returns = np.sort(returns)
    index = int((1 - confidence_level) * len(sorted_returns))
    VaR = abs(sorted_returns[index])
    CVaR = np.mean(sorted_returns[:index])

    return CVaR

def gaussian_VaR(returns,confidence_level):
    returns = np.array(returns)
    mean_return = np.mean(returns)
    std_dev = np.std(returns)
    z_score = np.abs(np.percentile(returns, (1 - confidence_level) * 100))
    var = mean_return - z_score * std_dev

    return var

def gaussian_CVaR(returns,confidence_level):

    returns = np.array(returns)
    mean_return = np.mean(returns)
    std_dev = np.std(returns)
    z_score = np.abs(np.percentile(returns, (1 - confidence_level) * 100))
    cvar = mean_return - (z_score * std_dev) / (1 - confidence_level)

    return cvar

def cornish_fisher_var(returns, confidence_level):
    
    returns = np.array(returns)
    mean_return = np.mean(returns)
    std_dev = np.std(returns)
    z_score = np.abs(np.percentile(returns, (1 - confidence_level) * 100))
    
    skewness = np.mean(((returns - mean_return) / std_dev) ** 3)
    kurtosis = np.mean(((returns - mean_return) / std_dev) ** 4)

    z_score_cornish_fisher = z_score + (z_score**2 - 1) * skewness / 6 + (z_score**3 - 3 * z_score) * kurtosis / 24 - (2 * z_score**3 - 5 * z_score) * skewness**2 / 36

    var = mean_return - z_score_cornish_fisher * std_dev

    return var